In [114]:
import pandas
from collections import defaultdict
import re
import datetime

In [115]:
#download turnstile data
#['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME','DESC', 'ENTRIES','EXITS']     

df = pandas.read_csv('turnstile_160618.txt')
temp = df.drop_duplicates('STATION',inplace = True)
stations = df[['STATION','LINENAME']]



In [53]:
#useful function for turning written numerals to numbers
def text2int(textnum, numwords={}):
    if not numwords:
        units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]
        
        ordinal_units = ["","first", 'second', 'third', 'fourth','fifth','sixth','sevent','eights','ninth']
        
        tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]
        
        scales = ["hundred", "thousand", "million", "billion", "trillion"]
        
        numwords["and"] = (1, 0)
        
        for idx, word in enumerate(units):
            numwords[word] = (1, idx)
        for idx, word in enumerate(ordinal_units):
            numwords[word] = (1, idx)
        for idx, word in enumerate(tens):
            numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales):
            numwords[word] = (10 ** (idx * 3 or 2), 0)

    current = result = 0
    for word in textnum.split():
        if word not in numwords:
            return(None)
            #raise Exception("Illegal word: " + word)

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0

    return result + current

print(text2int("twenty third"))
#7100031337

23


In [98]:
##http://web.mta.info/developers/data/nyct/subway/StationEntrances.csv
p = pandas.read_csv('StationEntrances.csv')

#['Division', 'Line', 'Station_Name', 'Station_Latitude',
#       'Station_Longitude', 'Route_1', 'Route_2', 'Route_3', 'Route_4',
#      'Route_5', 'Route_6', 'Route_7', 'Route_8', 'Route_9', 'Route_10',
#       'Route_11', 'Entrance_Type', 'Entry', 'Exit_Only', 'Vending',
#       'Staffing', 'Staff_Hours', 'ADA', 'ADA_Notes', 'Free_Crossover',
#       'North_South_Street', 'East_West_Street', 'Corner', 'Latitude',
#       'Longitude']###

station_gtfs = p[['Line', 'Station_Name', 'Station_Latitude','Station_Longitude']]

station_gtfs_nodupes = station_gtfs.drop_duplicates('Station_Name')
#station_info.drop_duplicates('NAME', inplace = True)



In [107]:
#match station name to coordinates from GTFS data
def match_station(s,pd_stn):
    
    for ind,row in pd_stn.iterrows():
        #change to upper case,infuse with .*
        w = row['Station_Name'].upper()
        temp = re.split(' |-|/',s)
        pattern ='.*'
        for v in temp:
            pattern = pattern + v + '.*'
        #match station name  
        if re.search(pattern,s):
            return((row.Station_Latitude,row.Station_Longitude))
    return(None)
    return(None)

In [113]:
#setting up stations to coordinates dictionary
st_coord = defaultdict(list)

#stations is df with columns STATION and LINE
#station_gtfs_nodupes is df with columns 'Line', 'Station_Name', 'Station_Latitude','Station_Longitude'

for ind, v in stations.iterrows():
    c = match_station(v.STATION,station_gtfs_nodupes)
    if c==None:
        print("No match for station", v.STATION)
    else:
        st_coord[v.STATION]=list(c)
        
#for k,v in st_coord.items():
    #print(k," : ",v)   

In [93]:
len(st_coord)

373